In [353]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import plotly.express as px


In [354]:
pandas_wiki_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_Supreme_Court_justices_by_time_in_office')[1]
date_errors = pandas_wiki_df[pd.to_datetime(pandas_wiki_df[pandas_wiki_df.columns[4]], errors="coerce").isna()]
date_errors
# pandas_wiki_df[pd.to_datetime(pandas_wiki_df[pandas_wiki_df.columns[4]], errors="coerce").isna()==False]

,Rank,Justice,Lengthin days,Lengthin years and days,Start date[3],End date[3]
7,8,William Rehnquist (CJ),"12,293[D]","33 years, 239 days","January 7, 1972 September 26, 1986","September 26, 1986 September 3, 2005"
23,24,Edward Douglass White (CJ),"9,929[E]","27 years, 68 days","March 12, 1894 December 19, 1910","December 19, 1910 May 19, 1921"
39,40,Harlan F. Stone (CJ),"7,721[F]","21 years, 51 days","March 2, 1925 July 3, 1941","July 2, 1941 April 22, 1946"
52,53,Charles Evans Hughes (CJ),"6,214[G]","17 years, 5 days","October 10, 1910 February 24, 1930","June 10, 1916 June 30, 1941"
112,113,John Rutledge (CJ),521[H],"1 year, 156 days","February 15, 1790 August 12, 1795","March 5, 1791 December 28, 1795"


In [355]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_Supreme_Court_justices_by_time_in_office'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find( "table", {"id":"justices"} )
# table

In [356]:
justices_tr = table.findAll("tr")[1:]

justice_name = []
justice_url = []
justice_term_start = []
justice_term_end = []

for justice in justices_tr:
    justice_str = justice.find_all('td')
    justice_name.append(justice_str[1].text)

    justiceurl = justice.find('a').get('href')
    justice_url.append(justiceurl)
    
    # justice_ts = justice.find_all('td')
    # justice_term_start.append(justice_ts[4].text)

    # justice_te = justice.find_all('td')
    # justice_term_end.append(justice_te[5].text)

justice_wiki_link_id = pd.DataFrame({
                                    'name': justice_name,
                                    'url': justice_url,
                    # 'term_start': justice_term_start,
                    # 'term_end':justice_term_end
                    })
justice_wiki_link_id 

,name,url
0,William O. Douglas,/wiki/William_O._Douglas
1,Stephen Johnson Field,/wiki/Stephen_Johnson_Field
2,John Paul Stevens,/wiki/John_Paul_Stevens
3,John Marshall (CJ),/wiki/John_Marshall
4,Hugo Black,/wiki/Hugo_Black
...,...,...
110,Robert Trimble,/wiki/Robert_Trimble
111,Amy Coney Barrett,/wiki/Amy_Coney_Barrett
112,John Rutledge (CJ),/wiki/John_Rutledge
113,Thomas Johnson,/wiki/Thomas_Johnson_(jurist)


In [357]:
urls = []
b_year = []
b_month = []
b_day = []
d_year = []
d_month = []
d_day = []


def get_birth_death(justice_url):
    try:
        url = 'https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles='+justice_url.replace("/wiki/", "")+'&format=xml'
        res = requests.get(url) 
        soup = BeautifulSoup(res.text, 'xml')
        
        urls.append(justice_url)
        # print(justice_url.replace("/wiki/", ""))

        birth_re = re.search(r'(birth_date(.*?)}})', soup.getText())
        birth_data = birth_re.group(0).split('|')
        birth_year = birth_data[1]
        b_year.append(birth_year)

        birth_month = birth_data[2]
        b_month.append(birth_month)

        birth_day = birth_data[3].replace(r"}}", "")
        b_day.append(birth_day)

        try:
            death_re = re.search(r'(death_date(.*?)}})', soup.getText())
            death_data = death_re.group(0).split('|')
            print(death_data)

            if death_data[0] > death_data[4]:
                death_year = death_data[1]
                d_year.append(death_year)

                death_month = death_data[2]
                d_month.append(death_month)

                death_day = death_data[3]
                d_day.append(death_day)
            else:
                death_year = death_data[4]
                d_year.append(death_year)

                death_month = death_data[5]
                d_month.append(death_month)

                death_day = death_data[6]
                d_day.append(death_day)
        except:
            d_year.append(np.nan)
            d_month.append(np.nan)
            d_day.append(np.nan)

    except:
        print(justice_url.replace("/wiki/", ""))
        b_year.append(np.nan)
        b_month.append(np.nan)
        b_day.append(np.nan)
        d_year.append(np.nan)
        d_month.append(np.nan)
        d_day.append(np.nan)

for url in justice_wiki_link_id['url']:
    get_birth_death(url)



['death_date = {{death date and age', '1980', '1', '19', '1898', '10', '16}}']
['death_date      = {{death date and age', '1899', '4', '9', '1816', '11', '4}}']
['death_date          = {{Death date and age', '2019', '07', '16', '1920', '04', '20}}']
['death_date = {{death date and age', '1835', '7', '6', '1755', '9', '24}}']
['death_date   = {{nowrap', '{{death date and age', '1971', '9', '25', '1886', '2', '27}}']
['death_date    = {{death date and age', '1911', '10', '14', '1833', '6', '1}}']
['death_date          = {{death date and age', '1997', '7', '24', '1906', '4', '25}}']
['death_date   = {{nowrap', '{{death date and age', '2005', '9', '3', '1924', '10', '1}}']
['death_date   = {{nowrap', '{{death date and age', '1845', '9', '10', '1779', '9', '18}}']
James_Moore_Wayne
['death_date   = {{death date and age', '1861', '4', '4', '1785', '3', '11}}']
['death_date          = {{death date and age', '2002', '4', '15', '1917', '6', '8}}']
['death_date          = {{nowrap', '{{death dat

In [358]:
birth_death_table = pd.DataFrame({
                                'urls': urls,
                                'b_year': b_year,
                                'b_month': b_month,
                                'b_day': b_day,
                                'd_year': d_year,
                                'd_month': d_month,
                                'd_day': d_day
                                })
birth_death_table['b_date'] = pd.to_datetime(birth_death_table['b_year']+"-"+birth_death_table['b_month']+"-"+birth_death_table['b_day'], errors="coerce")
birth_death_table['d_date'] = pd.to_datetime(birth_death_table['d_year']+"-"+birth_death_table['d_month']+"-"+birth_death_table['d_day'], errors="coerce")

# birth_death_table#['b_year'].unique()


In [359]:
workflow_combine_df = pandas_wiki_df.merge(justice_wiki_link_id ,how="left",left_on="Justice",right_on="name").merge(birth_death_table,how="left",left_on="url",right_on="urls")

In [360]:
formatted_df = pd.concat([workflow_combine_df[pd.to_datetime(workflow_combine_df[workflow_combine_df.columns[4]], 
                                            errors="coerce").isna()==False]])
formatted_df['start_date'] = pd.to_datetime(formatted_df[formatted_df.columns[4]], 
                                            errors="coerce")
formatted_df['end_date'] = pd.to_datetime(np.where(formatted_df[formatted_df.columns[5]]=="Incumbent",
                                    pd.to_datetime("today").strftime("%Y-%m-%d"),
                                    formatted_df[formatted_df.columns[5]]
                                    ))
formatted_df = formatted_df.sort_values(by="end_date", ascending=False)
# print(formatted_df.info())
# formatted_df[['start_date','end_date','b_date','d_date']]


In [361]:
# formatted_df

In [362]:
# formatted_df = formatted_df[['start_date','end_date']]

# fig, ax = plt.subplots(figsize=(40, 65))
# ax = ax.xaxis_date()
# ax = plt.hlines(formatted_df.index, formatted_df['start_date'], formatted_df['end_date'], linewidth=25)
# ax = plt.yticks(fontsize = 30)
# ax = plt.xticks(fontsize = 75)

In [363]:
plotly_df = formatted_df[['Justice','start_date','end_date','b_date','d_date']]
life = plotly_df[['Justice','b_date','d_date']].rename(columns={'b_date':'start_date','d_date':'end_date'})
life['task'] = 'life'
term = plotly_df[['Justice','start_date','end_date']]
term['task'] = 'term'
df = pd.concat([life,term]).dropna().sort_values(by="end_date", ascending=False)
df['start_date'] = df['start_date'].astype('datetime64')
df['end_date'] = df['end_date'].astype('datetime64')
# df[df['Justice']=="Antonin Scalia"]
df[df['start_date']>="1900-01-01"]

# term


/tmp/ipykernel_3798647/565521749.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Justice,start_date,end_date,task
111,Amy Coney Barrett,2020-10-27,2022-06-28,term
54,John Roberts (CJ),2005-09-29,2022-06-28,term
106,Brett Kavanaugh,2018-10-06,2022-06-28,term
97,Neil Gorsuch,2017-04-10,2022-06-28,term
21,Stephen Breyer,1994-08-03,2022-06-28,term
...,...,...,...,...
44,William R. Day,1903-03-02,1922-11-13,term
94,John Hessin Clarke,1916-10-09,1922-09-18,term
100,Joseph Rucker Lamar,1911-01-03,1916-01-02,term
103,Horace Harmon Lurton,1910-01-03,1914-07-12,term


In [365]:
colors = {}
colors['term'] =  'rgb(29, 133, 60)' #specify the color for the 'planned' schedule bars
colors['life'] = 'rgb(245, 148, 22)'  #specify the color for the 'actual' schedule bars

fig = px.timeline(
    df, 
    x_start="start_date", 
    x_end="end_date", 
    y="Justice",
    color='task',
    color_discrete_map = colors,
    # hover_name="Task Description"
    width=1200, height=3200
    )

fig.update_yaxes(autorange="reversed")          #if not specified as 'reversed', the tasks will be listed from bottom up       
fig.data[1].width=0.5 # update the width of the 'Actual' schedule bars (the second trace of the figure)
fig.show()
